### [GROUP BY](https://www.youtube.com/watch?v=9qrDsY_2COo&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=59)

In [3]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F
import pandas as pd

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('pyspark_test') \
    .getOrCreate()

In [3]:
df_green = spark.read.parquet(f'../../data/pq/green/*/*')

df_green.registerTempTable('green')

df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM green
WHERE lpep_pickup_datetime > "2020-01-01 00:00:00"
GROUP BY hour, zone
""")

df_green_revenue \
    .repartition(20) \
    .write.parquet('../../data/report/revenue/green', mode='overwrite')

/home/ramazan/spark/spark-3.4.2-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_yellow = spark.read.parquet(f'../../data/pq/yellow/*/*')

df_yellow.registerTempTable('yellow')

df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM yellow
WHERE tpep_pickup_datetime > "2020-01-01 00:00:00"
GROUP BY hour, zone
""")

df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../../data/report/revenue/yellow', mode='overwrite')

### [JOINS](https://www.youtube.com/watch?v=lu7TrqAWuH4&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=60)

In [7]:
# Use materialized tables, as opposed to generating them on-the-fly, 

df_green_revenue = spark.read.parquet('../../data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('../../data/report/revenue/yellow')

In [8]:
df_green_revenue_temp = df_green_revenue \
    .withColumnRenamed("amount", "green_amount") \
    .withColumnRenamed("number_records", "green_number_records")

df_yellow_revenue_temp = df_yellow_revenue \
    .withColumnRenamed("amount", "yellow_amount") \
    .withColumnRenamed("number_records", "yellow_number_records")

In [9]:
df_total = df_green_revenue_temp.join(df_yellow_revenue_temp, on=["hour", "zone"], how="outer")

In [10]:
df_total.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [11]:
df_total.write.parquet('../../data/report/revenue/total', mode="overwrite")

In [12]:
df_total = spark.read.parquet('../../data/report/revenue/total')

In [13]:
df_total.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  25| 531.0000000000002|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  43|            107.52|                   6| 6539.510000000012|                  390|
|2020-01-01 00:00:00|  49|26

In [14]:
df_zones = spark.read.parquet('../../data/pq/zones')

In [15]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [17]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [27]:
df_result = df_total.join(df_zones, df_total.zone == df_zones.LocationID)
df_result = df_result.drop('LocationID', 'zone')
df_result.show(1, vertical=True)

-RECORD 0------------------------------------
 hour                  | 2020-01-01 00:00:00 
 green_amount          | null                
 green_number_records  | null                
 yellow_amount         | 8.8                 
 yellow_number_records | 1                   
 Borough               | Brooklyn            
 service_zone          | Boro Zone           
only showing top 1 row



In [28]:
df_result.write.parquet('../../data/pq/tmp/revenue_zones')